In [ ]:
#SNN : shared Neirest Neighbors (clustering basé graphe)
#***********************************
#       UTILE POUR LE RAPPORT
#***********************************

# graphe de voisinage
#(DBSCAN, clustering basé densité)
# cours :https://homepages.laas.fr/huguet/drupal/sites/homepages.laas.fr.huguet/files/u78/2018-2019-cours_AP-NS-Part-2-Clustering.pdf 

# SNN et DBSCAN permet de gérer des densité de points différentes, de trouver des formes complexes

#Algorithme SNN-DBSCAN
#1. Calculer la matrice de similarité point à point
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires
#3.Construire le graphe SNN
#    Appliquer un seuil de similarité sur la matrice 
#   appliquer une recherche de composantes connexes pour obtenir les clusters
#4.Appliquer le principe de DBSCAN
#   Paramètres : epsilon et MinPts
#     Calculer le epsilon-voisinage de chaque point  x
#     densité SNN(x)
#     Déterminer les points intérieurs (voisinage au moins de taille MinPts) et construire les clusters associés
#     Retirer les points atypiques86


# Variante : Jarvis-Patrick 
#1.Calculer la matrice de similarité point à point
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires
#3.Construire le graphe SNN
#  


#   Méthode ChameleonPré-processing : 
#   Calculer le graphe des plus proches voisins
#   Deux phases
#   Déterminer des sous clusters initiaux (partition du graphe des k-ppv)
#   Approche hiérarchique ascendante : 
        #Mesures spécifiques pour regrouper des clusters (inter-connectivité et proximité)88


In [2]:
# Imports nécessaires
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from sklearn.neighbors import kneighbors_graph






In [3]:
# 1. CONSTRUIRE LA MATRICE DE SIMILARITE
# SIMILARITE : 1/(1+distance)

data1 = np.loadtxt( 'cham-data/t4.8k.dat' )
# Autre possibilité, genloadstxt
data1[1][0]
print(' shape data1 : ' , data1.shape)
print(data1[1,])
# chaque ligne représente les coordonnées d'un point


def similarity_func(u, v):
    return 1/(1+euclidean(u,v))

def similarity_Matrix(data):
    numPoints = len(data)
    matrix = np.zeros(shape=(numPoints, numPoints))
    for i in range(len(matrix)):
        for j in range(i+1,len(matrix)):
            sim = similarity_func(data[i], data[j])
            matrix[i,j] = sim
    return matrix

# Similarity matrix OK

 shape data1 :  (8000, 2)
[454.665985 264.80899 ]


In [15]:
#test Similarity_Matrix
data=([
    [1, 0],
    [1, 1],
    [2, 1]])
#similarity_Matrix(data)
sim1=similarity_Matrix(data)
#simtot=similarity_Matrix(data1)

In [16]:
#test
sim1
#simtot

array([[0.        , 0.5       , 0.41421356],
       [0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        ]])

In [11]:
# TESTS sur SIM1
# par défaut, le kneighbor graph compare les distances euclidiennes, 
#et renvoit les plus proches voisins de chaque point (sous forme de couple).
# On peut setter la distance . 
# EN ENTREE :
# - on peut lui donner les données brutes
# - on peut aussi lui donner la matrice de similarité
#DOC ICI : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html
#MJ dit : c'est OK pour l'utiliser. le but étant de faire un meilleur score que DBSCAN
A = kneighbors_graph(sim1, 3, mode='connectivity', include_self= False).toarray()

#output, matrice avec les 2 plus proches voisins et les distance euclidienne
# on fait la distance euclidienne de la matrice de similarité
B = kneighbors_graph(sim1, 3, mode='distance', include_self= False).toarray()

# idem mais ici, distance euclidienne des points
C = kneighbors_graph(data1[1:8,:], 3, mode='distance', include_self= False).toarray()

#mutilication de la matrice de similarité par la matrice de connectivité
D= A*sim1

#TODO : trouver les paramètres optimaux et comment appliquer la suite de l'algo.
# idée : se servir de A pour mettre à jour la matrice de similarité
print("A, input similarity matrice, mode connectivity \n",A, "\n")
print("B, input similarity matrice, mode distance \n",B,"\n")
print("C, input data (x,y of each point), mode distance\n", C, "\n")

print("Sim1, print de la matrice de similarité\n",sim1,"\n")
print("D, A*sim1, connectivité * similarity \n",D)


A, input similarity matrice, mode connectivity 
 [[0. 0. 1. 1. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]] 

B, input similarity matrice, mode distance 
 [[0.         0.         0.01240566 0.0135567  0.         0.
  0.01365098]
 [0.02686166 0.         0.02456096 0.02382791 0.         0.
  0.        ]
 [0.         0.         0.         0.00549448 0.00745487 0.
  0.00705449]
 [0.         0.         0.00549448 0.         0.00421057 0.
  0.00849167]
 [0.         0.         0.00745487 0.00421057 0.         0.
  0.00908363]
 [0.         0.         0.07452578 0.07207866 0.07131507 0.
  0.        ]
 [0.         0.         0.00705449 0.00849167 0.00908363 0.
  0.        ]] 

C, input data (x,y of each point), mode distance
 [[  0.           0.          82.06794747 266.92960789 358.6524946
    0.           0.        ]
 [  0.           0.           0.           0.          36.4929661
  1

In [13]:
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires

# par défaut, le kneighbor graph compare les distances euclidiennes, 
#et renvoit les plus proches voisins de chaque point (sous forme de couple).
# On peut setter la distance . 
# EN ENTREE :
# - on peut lui donner les données brutes
# - on peut aussi lui donner la matrice de similarité

def filtrage_simMatrix(sim, k):
    # recuperer les k plus proches voisins ( matrice de 0 et de 1)
    A = kneighbors_graph(sim, k, mode='connectivity', include_self= False).toarray()
    # On actualise la matrice de similarité en mettant à 0 les voisins qui ne sont pas proches
    SimVoisin= sim*A
    return SimVoisin
    

In [21]:
# test
simVoisin=filtrage_simMatrix(sim1,3)
simVoisin
# NOTE : c'est anormalement longpour la matrice sur Simtot (8000 points...)

array([[0.        , 0.        , 0.01203834, 0.00373232, 0.        ,
        0.        , 0.00249365],
       [0.        , 0.        , 0.00347096, 0.00424803, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00436448, 0.0035881 ,
        0.        , 0.00299362],
       [0.        , 0.        , 0.        , 0.        , 0.00389552,
        0.        , 0.00546045],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00623654],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [27]:
#3. Créer le grape SNN
# pour générer le graphe SNN, il faut compter le nombre de voisins en commun puis mettre le poids
    # l'algo se trouve p11 de ce pdf : https://pdfs.semanticscholar.org/fc0a/31e27925b62cf5931aab23995644d4d6dd3e.pdf
    
def SNNmatrix(nnmatrix, k):
    snnmatrix= np.zeros((len(nnmatrix), len(nnmatrix)))
    for i in range(len(nnmatrix)):
        for j in range(i+1, len(nnmatrix)):
            counter=0
            for m in range(len(nnmatrix)):
                if nnmatrix[i][m]!=0. and nnmatrix[m][j]!=0. :
                    counter +=1
            snnmatrix[i][j]=counter # c'est pas pour la version finale ...
            # TODO :
            #if counter>=k
                #connecter l'arc pondéré
    return snnmatrix

# TODO : testé, mais ça ne donne pas le bon résultat.
# A Débuguer sur un exemple

In [29]:
snn=SNNmatrix(simVoisin,1)
snn

array([[0., 0., 0., 1., 2., 0., 2.],
       [0., 0., 0., 1., 2., 0., 2.],
       [0., 0., 0., 0., 1., 0., 2.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
#4. Appliquer le principe de DBSCAN
# J'ai pas trop saisi coment faire ...